In [1]:
# Agrege acá el código para importar las librerias

# La libreria para "encontrar el sevicio" de Spark
import findspark
findspark.init()

# Librerias para "gestionar el servicio" de Spark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession

# Creamos una aplicación Spark en el Servicio
# Tenga cuidado con las tildes o caracteres especiales en el nombre de la app
AppSpark = SparkConf().setAppName("Evaluacion iSofware")

# definimos un espacio o contexto para la App
ContextoSpark=SparkContext(conf=AppSpark)

# inicio una sesión en el espacio de la App
SesionSpark = SparkSession(ContextoSpark)

# inicio del espacio o contexto SQL
ContextoSql = SQLContext(sparkContext=ContextoSpark, sparkSession=SesionSpark)

In [2]:
!hadoop fs -ls

In [89]:
#!hdfs dfs -rm /tmp/Asistencia1.csv
#!hdfs dfs -rm /tmp/Asistencia2.csv
#!hdfs dfs -rm /tmp/Evaluacion.csv

Deleted /tmp/Evaluacion.csv


In [3]:
!hdfs dfs -copyFromLocal  Asistencia.csv /tmp/

In [5]:
!hdfs dfs -copyFromLocal  Evaluacion.csv /tmp/

copyFromLocal: `/tmp/Evaluacion.csv': File exists


In [6]:
!hdfs dfs -ls /tmp/*csv

-rw-r--r--   1 root supergroup      93121 2021-05-10 02:44 /tmp/Asistencia.csv
-rw-r--r--   1 root supergroup     111944 2021-05-10 02:44 /tmp/Evaluacion.csv


In [ ]:
# 2. Cree dos tablas SparkSQL y almacene el csv en su correspondiente tabla.
# Observación: tenga especial cuidado con los encabezados de los archivos CSV.
# Usted puede considerar cambiar los encabezados de los CSV originales
# Tu código a continuación...

In [11]:
# Creamos una tabla a partir de datos CSV
# Se toma el CSV desde el FS de Hadoop
#https://www.learningjournal.guru/courses/spark/spark-foundation-training/spark-sql-database-and-table/
#https://spark.apache.org/docs/latest/sql-ref-syntax-ddl.html

ContextoSql.sql("""
CREATE TABLE 
    Asistencia 
USING com.databricks.spark.csv 
OPTIONS (
    path 'Asistencia.csv', 
    header 'true')
""");

AnalysisException: 'Unable to infer schema for CSV. It must be specified manually.;'

In [7]:
Asistencia = ContextoSql.read.load('/tmp/Asistencia.csv',
                                   format="csv",
                                   sep=',',
                                   inferSchema='true',
                                   header='true')

In [8]:
Asistencia = Asistencia.withColumnRenamed("Marca temporal","Marca")\
.withColumnRenamed("Nombre de usuario","Nombre")\
.withColumnRenamed("Equipo al que perteneces:","Equipo_perteneces")\
.withColumnRenamed("Equipo que va a exponer:","Equipo_exponer")

Asistencia.printSchema()

root
 |-- Marca: string (nullable = true)
 |-- Nombre: string (nullable = true)
 |-- Equipo_perteneces: string (nullable = true)
 |-- Equipo_exponer: string (nullable = true)



In [9]:
Evaluacion = ContextoSql.read.load('/tmp/Evaluacion.csv',
                                   format="csv",
                                   sep=',',
                                   inferSchema='true',
                                   header='true')

In [10]:
Evaluacion.printSchema()

root
 |-- Marca temporal: string (nullable = true)
 |-- Nombre de usuario: string (nullable = true)
 |-- Equipo que vas a evaluar:: string (nullable = true)
 |-- Introducción: El equipo responde adecuadamente ¿Quiénes son y por qué están aquí?: integer (nullable = true)
 |-- Equipo: El equipo responde adecuadamente ¿Quiénes están detrás de la idea y cuál es su función?: integer (nullable = true)
 |-- Problema: El equipo responde adecuadamente ¿Qué problema resolverá?, ¿es realmente un problema?: integer (nullable = true)
 |-- Ventajas: El equipo responde adecuadamente ¿Por qué su solución es especial?, ¿qué la hace distinta de otras?: integer (nullable = true)
 |-- Solución: El equipo responde adecuadamente ¿Cómo piensa resolver el problema?: integer (nullable = true)
 |-- Producto: El equipo responde adecuadamente ¿Cómo funciona el producto o servicio? Muestra algunos ejemplos.: integer (nullable = true)
 |-- Tracción: El equipo responde adecuadamente si cuenta con clientes que demues

In [11]:
Evaluacion = Evaluacion.withColumnRenamed("Marca temporal","Marca")\
.withColumnRenamed("Nombre de usuario","Nombre")\
.withColumnRenamed("Equipo que vas a evaluar:","Equipo_evaluar")\
.withColumnRenamed("Introducción: El equipo responde adecuadamente ¿Quiénes son y por qué están aquí?","Introduccion")\
.withColumnRenamed("Equipo: El equipo responde adecuadamente ¿Quiénes están detrás de la idea y cuál es su función?","Equipo")\
.withColumnRenamed("Problema: El equipo responde adecuadamente ¿Qué problema resolverá?, ¿es realmente un problema?","Problema")\
.withColumnRenamed("Ventajas: El equipo responde adecuadamente ¿Por qué su solución es especial?, ¿qué la hace distinta de otras?","ventajas")\
.withColumnRenamed("Solución: El equipo responde adecuadamente ¿Cómo piensa resolver el problema?","solucion")\
.withColumnRenamed("Producto: El equipo responde adecuadamente ¿Cómo funciona el producto o servicio? Muestra algunos ejemplos.","Producto")\
.withColumnRenamed("Tracción: El equipo responde adecuadamente si cuenta con clientes que demuestran potencial.","Traccion")\
.withColumnRenamed("Mercado: El equipo responde conoce, o por lo menos intentar predecir, el tamaño del mercado que impactará.","Mercado")\
.withColumnRenamed("Competencia: El equipo responde adecuadamente ¿Cuáles son las soluciones alternativas al problema que plantea?","Competencia")\
.withColumnRenamed("Modelo de negocio: El equipo responde adecuadamente ¿Cómo hará dinero? ","Modelo_neg")\
.withColumnRenamed("Inversión: El equipo responde adecuadamente ¿Cuál es su presupuesto y cuánto espera ganar?","inversion")\
.withColumnRenamed("Contacto: El equipo deja los datos al cliente y muestra cómo pueden contactarle.","contacto")\
.withColumnRenamed("Exposición: ¿Qué tan coordinados estaban los expositores?","Coordinacion_expo")\
.withColumnRenamed("Exposición: ¿Los expositores se expresaron con claridad y se hicieron entender?","Expo_claridad")\
.withColumnRenamed("Exposición: Las diapositivas son claras y coherentes y apoyaron adecuadamente la exposición.","Diapo_clara")\
.withColumnRenamed("Suponiendo que eres inversionista, ¿Estarías dispuesto a invertir dinero en este equipo? (esta pregunta no se pondera en la nota)","Inversionista")\
.withColumnRenamed("Observaciones para el equipo, estas observaciones las debe considerar el equipo para mejorar la siguiente presentación.","Observacion")



Evaluacion.printSchema()

root
 |-- Marca: string (nullable = true)
 |-- Nombre: string (nullable = true)
 |-- Equipo_evaluar: string (nullable = true)
 |-- Introduccion: integer (nullable = true)
 |-- Equipo: integer (nullable = true)
 |-- Problema: integer (nullable = true)
 |-- ventajas: integer (nullable = true)
 |-- solucion: integer (nullable = true)
 |-- Producto: integer (nullable = true)
 |-- Traccion: integer (nullable = true)
 |-- Mercado: integer (nullable = true)
 |-- Competencia: integer (nullable = true)
 |-- Modelo_neg: integer (nullable = true)
 |-- inversion: integer (nullable = true)
 |-- contacto: integer (nullable = true)
 |-- Coordinacion_expo: integer (nullable = true)
 |-- Expo_claridad: integer (nullable = true)
 |-- Diapo_clara: integer (nullable = true)
 |-- Inversionista: string (nullable = true)
 |-- Observacion: string (nullable = true)



In [ ]:
# 2. Consulte el listado total de estudiantes (correos electrónicos) del 
# curso de Ingeniería de Software, ordenados alfabéticamente
# Tu código a continuación...

In [12]:
Asistencia.createOrReplaceTempView('VistaAsistencia')

In [13]:
ContextoSql.sql("""
select distinct Nombre
from VistaAsistencia
order by Nombre
""").show(10)

+--------------------+
|              Nombre|
+--------------------+
| abanolc@unal.edu.co|
|acastrillonv@unal...|
| aflemag@unal.edu.co|
|anapariciom@unal....|
|angutierrezb@unal...|
|anoriega@unal.edu.co|
|auarbelaeza@unal....|
| bocampo@unal.edu.co|
|cgiraldo@unal.edu.co|
|cjfunezg@unal.edu.co|
+--------------------+
only showing top 10 rows



In [ ]:
# 3. Consulte la cantidad de asistencias registradas por estudiante; además, la fecha y hora de la primera asistencia
# y la fecha y hora de la última asistencia
# Tu código a continuación...

In [14]:
ContextoSql.sql("""
select Nombre, Count(Marca) as cantidad_asist, min(Marca) as primera_asistencia, max(Marca) as ultima_asistencia
from VistaAsistencia
group by Nombre
order by Nombre
""").show(10)

+--------------------+--------------+--------------------+--------------------+
|              Nombre|cantidad_asist|  primera_asistencia|   ultima_asistencia|
+--------------------+--------------+--------------------+--------------------+
| abanolc@unal.edu.co|             7|2021/04/08 8:12:1...|2021/04/13 9:37:5...|
|acastrillonv@unal...|             8|2021/04/08 8:13:0...|2021/04/13 9:43:1...|
| aflemag@unal.edu.co|             8|2021/04/08 8:11:5...|2021/04/13 9:41:5...|
|anapariciom@unal....|            10|2021/04/08 8:11:4...|2021/04/13 9:38:5...|
|angutierrezb@unal...|             8|2021/04/08 8:19:0...|2021/04/13 9:43:5...|
|anoriega@unal.edu.co|             8|2021/04/08 8:15:1...|2021/04/13 9:40:2...|
|auarbelaeza@unal....|             8|2021/04/08 8:09:5...|2021/04/13 9:41:0...|
| bocampo@unal.edu.co|             8|2021/04/08 8:13:3...|2021/04/13 9:42:1...|
|cgiraldo@unal.edu.co|             8|2021/04/08 8:44:2...|2021/04/13 9:56:4...|
|cjfunezg@unal.edu.co|             8|202

In [15]:
# 4. Consulte el listado de estudiantes que asistieron a 2 presentaciones o menos (una).
# Tu código a continuación...

ContextoSql.sql("""
select Nombre, Count(Marca) as cantidad_asist 
from VistaAsistencia
group by Nombre
having cantidad_asist>0 AND cantidad_asist<=2
order by Nombre
""").show(10)

+--------------------+--------------+
|              Nombre|cantidad_asist|
+--------------------+--------------+
|jdroldano@unal.ed...|             1|
+--------------------+--------------+



In [24]:
# 5. Consulte el listado de estudiantes que no asistieron a ninguna presentación.
# Tu código a continuación...

Evaluacion.createOrReplaceTempView('VistaEvaluacion')

# nombres evaluacion sin repetir = Todos los del curso
# - nombres asistieron

# ------------

# Personas que no asistieron


#FALTA

In [98]:
ContextoSql.sql("""
select Nombre 
from VistaEvaluacion
order by Nombre DESC
""").show(10)

+-------------------+
|             Nombre|
+-------------------+
|wocampo@unal.edu.co|
|wocampo@unal.edu.co|
|wocampo@unal.edu.co|
|wocampo@unal.edu.co|
|wocampo@unal.edu.co|
|wocampo@unal.edu.co|
|wocampo@unal.edu.co|
|  tleon@unal.edu.co|
|  tleon@unal.edu.co|
|  tleon@unal.edu.co|
+-------------------+
only showing top 10 rows



In [ ]:
# 6. Consulte los integrantes por cada equipo al que pertenecen.
# Tu código a continuación...

In [17]:
ContextoSql.sql("""
select distinct Nombre, Equipo_perteneces
from VistaAsistencia
order by Equipo_perteneces, Nombre
""").show()

+--------------------+--------------------+
|              Nombre|   Equipo_perteneces|
+--------------------+--------------------+
|angutierrezb@unal...|Pertenezco a: Adm...|
| eapenad@unal.edu.co|Pertenezco a: Adm...|
|emflorezb@unal.ed...|Pertenezco a: Adm...|
|framirezf@unal.ed...|Pertenezco a: Adm...|
|jhcordoba@unal.ed...|Pertenezco a: Adm...|
| jortize@unal.edu.co|Pertenezco a: Adm...|
| kmolano@unal.edu.co|Pertenezco a: Adm...|
|lumarinb@unal.edu.co|Pertenezco a: Adm...|
|nsalinas@unal.edu.co|Pertenezco a: Adm...|
|rmbuilesm@unal.ed...|Pertenezco a: Adm...|
|saespinosab@unal....|Pertenezco a: Adm...|
|samolinap@unal.ed...|Pertenezco a: Adm...|
|sednarvaezna@unal...|Pertenezco a: Adm...|
| abanolc@unal.edu.co|Pertenezco a: Ges...|
|anapariciom@unal....|Pertenezco a: Ges...|
|davgarciava@unal....|Pertenezco a: Ges...|
|emlopezr@unal.edu.co|Pertenezco a: Ges...|
|favasquezj@unal.e...|Pertenezco a: Ges...|
|hriveraa@unal.edu.co|Pertenezco a: Ges...|
|jcardonaso@unal.e...|Pertenezco

In [18]:
ContextoSql.sql("""
select Equipo_perteneces, Count(Nombre) as cantidad_integrantes
from (
select Distinct Nombre, Equipo_perteneces
from VistaAsistencia
)
group by Equipo_perteneces
order by Equipo_perteneces
""").show()

+--------------------+--------------------+
|   Equipo_perteneces|cantidad_integrantes|
+--------------------+--------------------+
|Pertenezco a: Adm...|                  13|
|Pertenezco a: Ges...|                  11|
|Pertenezco a: Ges...|                  12|
|Pertenezco a: Ges...|                   9|
|Pertenezco a: Lo ...|                  11|
|Pertenezco a: Mi ...|                  10|
|Pertenezco a: Ser...|                  10|
|Pertenezco a: Te ...|                  11|
+--------------------+--------------------+



In [ ]:
# 7. Consulte la cantidad de asistentes por presentación, sin considerar los asistentes que pertenecen 
# al equipo que realizó la presentación.
# Tu código a continuación...

In [19]:
ContextoSql.sql("""
select Equipo_exponer, Count(Nombre) as Cantidad_Asistente
from VistaAsistencia
where substring(Equipo_exponer,9,4) != substring(Equipo_perteneces,15,4)
group by Equipo_exponer
order by Equipo_exponer
""").show()


+--------------------+------------------+
|      Equipo_exponer|Cantidad_Asistente|
+--------------------+------------------+
|Expone: Administr...|                70|
|Expone: Deserción...|                 3|
|Expone: Gestión d...|                52|
|Expone: Gestión d...|                53|
|Expone: Gestión d...|                52|
|Expone: Lo tengo ...|                75|
|Expone: Mi Sanand...|                72|
|Expone: Servicios...|                76|
|Expone: Te lo cam...|                75|
+--------------------+------------------+



In [26]:
ContextoSql.sql("""
select Equipo_evaluar
from VistaEvaluacion
""").show()

#, Count(Nombre) as Cantidad_Asistente
#where substring(Equipo_exponer,9,4) != substring(Equipo_perteneces,15,4)
#group by Equipo_exponer
#order by Equipo_exponer

+--------------------+
|      Equipo_evaluar|
+--------------------+
|       Te lo cambio.|
|       Te lo cambio.|
|       Te lo cambio.|
|       Te lo cambio.|
|       Te lo cambio.|
|       Te lo cambio.|
|       Te lo cambio.|
|       Te lo cambio.|
|       Te lo cambio.|
|Lo tengo y te lo ...|
|       Te lo cambio.|
|       Te lo cambio.|
|       Te lo cambio.|
|       Te lo cambio.|
|       Te lo cambio.|
|       Te lo cambio.|
|       Te lo cambio.|
|       Te lo cambio.|
|       Te lo cambio.|
|       Te lo cambio.|
+--------------------+
only showing top 20 rows



In [ ]:
# 8. Consutar cuáles integrantes evaluaron a su propio equipo. Estas evaluaciones no serán válidas, pues un
# integrante no puede evaluar a su propio equipo.
# Tu código a continuación...

In [98]:
ContextoSql.sql("""

WITH 

ASISTENCIAS AS (
select
Equipo_exponer, 
Equipo_perteneces,
Nombre,
substring(Equipo_perteneces,15,15) as llave_equipo_pertenece,
substring(Equipo_exponer,9,15) as llave_equipo_exponer
from VistaAsistencia
),   

EVALUACION AS (
select 
Nombre,
substring(Equipo_evaluar,1,15) as llave_Equipo_a_evaluar,
Equipo_evaluar
from VistaEvaluacion
)

    select distinct b1.Nombre,
    b1.llave_equipo_pertenece, 
    b2.llave_Equipo_a_evaluar, 
    count(*) as Num_veces_evaluo_equipo
    from ASISTENCIAS b1
    inner join EVALUACION b2 on b1.Nombre = b2.Nombre  AND  b1.llave_equipo_pertenece = b2.llave_Equipo_a_evaluar
    group by b1.Nombre, b1.llave_equipo_pertenece, b2.llave_Equipo_a_evaluar

""").show()

+--------------------+----------------------+----------------------+-----------------------+
|              Nombre|llave_equipo_pertenece|llave_Equipo_a_evaluar|Num_veces_evaluo_equipo|
+--------------------+----------------------+----------------------+-----------------------+
|hriveraa@unal.edu.co|       Gestión de Menú|       Gestión de Menú|                      7|
|jhcordoba@unal.ed...|       Administración |       Administración |                      1|
|daestradam@unal.e...|       Lo tengo y te l|       Lo tengo y te l|                      8|
| jruedat@unal.edu.co|       Servicios a dom|       Servicios a dom|                      8|
| kmolano@unal.edu.co|       Administración |       Administración |                      1|
|  lmayar@unal.edu.co|       Gestión de prog|       Gestión de prog|                      9|
|mcarvajalsa@unal....|       Gestión de Resi|       Gestión de Resi|                      1|
|anapariciom@unal....|       Servicios a dom|       Servicios a dom|  

In [ ]:
# 9. Consultar la nota promedio por cada ítem (1 al 15), y la nota promedio total del cada equipo. Recuerde que no
# son válidas las evaluaciones realizadas por los miembros del mismo equipo.
# Tu código a continuación...

#Nota: Hay evaluaciones de nombre usuario repetida, ¿Cúal de las evaluaciones se debe escoger? ¿Ambas? --- Tienen diferente hora y calificación


In [37]:
ContextoSql.sql("""

WITH 

ASISTENCIAS AS (
select *,
substring(Equipo_perteneces,15,15) as llave_equipo_pertenece,
substring(Equipo_exponer,9,15) as llave_equipo_exponer
from VistaAsistencia
),

EVALUACION AS (
select *,
substring(Equipo_evaluar,1,15) as llave_Equipo_a_evaluar
from VistaEvaluacion
),

ASISTENTES_VERDADERAS AS (
select 
b1.Nombre,
b1.Equipo_perteneces,
b2.*

from ASISTENCIAS b1 
inner join EVALUACION b2 on b1.Nombre = b2.Nombre
where b1.llave_equipo_pertenece != b2.llave_Equipo_a_evaluar 
)

    select Equipo_evaluar,
    ROUND(AVG(Introduccion),2) as a, 
    ROUND(AVG(Equipo),2) as b, 
    ROUND(AVG(Problema),2) as c, 
    ROUND(AVG(ventajas),2) as d, 
    ROUND(AVG(Solucion),2) as e,
    ROUND(AVG(Producto),2) as f,
    ROUND(AVG(Traccion),2) as g,
    ROUND(AVG(Mercado),2) as h,
    ROUND(AVG(Competencia),2) as i,
    ROUND(AVG(Modelo_neg),2) as j,
    ROUND(AVG(inversion),2) as k,
    ROUND(AVG(contacto),2) as l,
    ROUND(AVG(Coordinacion_expo),2) as m, 
    ROUND(AVG(Expo_claridad),2) as n, 
    ROUND(AVG(Diapo_clara),2) as o,
    ROUND(AVG(Introduccion+Equipo+Problema+ventajas+Solucion+Producto+Traccion+Mercado+Competencia+Modelo_neg+inversion+contacto+Coordinacion_expo+Expo_claridad+Diapo_clara)/15,2) AS PROMEDIO_TOTAL

    from ASISTENTES_VERDADERAS
    group by Equipo_evaluar
    order by Equipo_evaluar

""").show()

+--------------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+--------------+
|      Equipo_evaluar|   a|   b|   c|   d|   e|   f|   g|   h|   i|   j|   k|   l|   m|   n|   o|PROMEDIO_TOTAL|
+--------------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+--------------+
|Administración de...|4.12|4.41|3.97|3.93|4.19|4.44|4.05|4.42|3.95|4.42|4.13|4.13|4.32|4.22|4.14|          4.19|
|Deserción de estu...| 4.5| 5.0| 5.0| 5.0| 4.5| 5.0| 5.0| 4.5| 5.0| 4.5| 4.5| 5.0| 4.5| 4.5| 5.0|          4.77|
|Gestión de Menús ...|4.32|3.98|4.57|4.32|4.38|4.02| 4.2|4.22| 4.3|4.64|4.31|4.51|4.46|4.45|4.73|          4.36|
|Gestión de Reside...|4.56|4.43|4.64|4.45|4.46|4.12|4.52|4.68|4.42|4.53|4.35|4.65|4.43|4.52|4.41|          4.48|
|Gestión de progra...|4.38| 4.4|4.55|3.92|4.34|4.45|4.42| 4.5|4.28|4.32|4.54|4.43|4.45|4.49|4.63|          4.41|
|Lo tengo y te lo ...|4.49| 4.6|4.27|4.43|4.42|4.72|4.36|4.25|4.49|4.47|4.36|4.59|4.53| 4.6|4.61

In [ ]:
# 10. Consulte el mejor equipo evaluado por cada ítem (según nota promedio. En caso de empate mostrar todos los empatados) 
# y el mejor equipo según el promedio total.
# Tu código a continuación...

In [ ]:
# 11. Consulte el peor equipo evaluado por cada ítem (según nota promedio. En caso de empate mostrar todos los empatados) 
# y el peor equipo según el promedio total.
# Tu código a continuación...

In [103]:
ContextoSql.sql("""

WITH 

ASISTENCIAS AS (
select *,
substring(Equipo_perteneces,15,15) as llave_equipo_pertenece,
substring(Equipo_exponer,9,15) as llave_equipo_exponer
from VistaAsistencia
),

EVALUACION AS (
select *,
substring(Equipo_evaluar,1,15) as llave_Equipo_a_evaluar
from VistaEvaluacion
),

ASISTENTES_VERDADERAS AS (
select 
b1.Nombre,
b1.Equipo_perteneces,
b2.*
from ASISTENCIAS b1 
inner join EVALUACION b2 on b1.Nombre = b2.Nombre
where b1.llave_equipo_pertenece != b2.llave_Equipo_a_evaluar 
),

PROMEDIOS AS(
select Equipo_evaluar,
ROUND(AVG(Introduccion),2) as a, 
ROUND(AVG(Equipo),2) as b, 
ROUND(AVG(Problema),2) as c, 
ROUND(AVG(ventajas),2) as d, 
ROUND(AVG(Solucion),2) as e,
ROUND(AVG(Producto),2) as f,
ROUND(AVG(Traccion),2) as g,
ROUND(AVG(Mercado),2) as h,
ROUND(AVG(Competencia),2) as i,
ROUND(AVG(Modelo_neg),2) as j,
ROUND(AVG(inversion),2) as k,
ROUND(AVG(contacto),2) as l,
ROUND(AVG(Coordinacion_expo),2) as m, 
ROUND(AVG(Expo_claridad),2) as n, 
ROUND(AVG(Diapo_clara),2) as o,
ROUND(AVG(Introduccion+Equipo+Problema+ventajas+Solucion+Producto+Traccion+Mercado+Competencia+Modelo_neg+inversion+contacto+Coordinacion_expo+Expo_claridad+Diapo_clara)/15,2) AS PROMEDIO_TOTAL
from ASISTENTES_VERDADERAS
group by Equipo_evaluar
order by Equipo_evaluar
)

    select [Te lo cambio.], [Lo tengo y te lo presto.],[Gestión de programas curriculares],[Administración de Inmuebles],[Servicios a domicilio],[Gestión de Menús Ejecutivos],[Gestión de programas curriculares],[Deserción de estudiantes],[Gestión de Residencias y Habitaciones Universitarias.]
    from PROMEDIOS
    pivot
        (
        MIN(a)
        for Equipo_evaluar IN ([Te lo cambio.], [Lo tengo y te lo presto.],[Gestión de programas curriculares],[Administración de Inmuebles],[Servicios a domicilio],[Gestión de Menús Ejecutivos],[Gestión de programas curriculares],[Deserción de estudiantes],[Gestión de Residencias y Habitaciones Universitarias.])
        ) as PV

""").show()

ParseException: "\nno viable alternative at input 'select ['(line 51, pos 11)\n\n== SQL ==\n\n\nWITH \n\nASISTENCIAS AS (\nselect *,\nsubstring(Equipo_perteneces,15,15) as llave_equipo_pertenece,\nsubstring(Equipo_exponer,9,15) as llave_equipo_exponer\nfrom VistaAsistencia\n),\n\nEVALUACION AS (\nselect *,\nsubstring(Equipo_evaluar,1,15) as llave_Equipo_a_evaluar\nfrom VistaEvaluacion\n),\n\nASISTENTES_VERDADERAS AS (\nselect \nb1.Nombre,\nb1.Equipo_perteneces,\nb2.*\nfrom ASISTENCIAS b1 \ninner join EVALUACION b2 on b1.Nombre = b2.Nombre\nwhere b1.llave_equipo_pertenece != b2.llave_Equipo_a_evaluar \n),\n\nPROMEDIOS AS(\nselect Equipo_evaluar,\nROUND(AVG(Introduccion),2) as a, \nROUND(AVG(Equipo),2) as b, \nROUND(AVG(Problema),2) as c, \nROUND(AVG(ventajas),2) as d, \nROUND(AVG(Solucion),2) as e,\nROUND(AVG(Producto),2) as f,\nROUND(AVG(Traccion),2) as g,\nROUND(AVG(Mercado),2) as h,\nROUND(AVG(Competencia),2) as i,\nROUND(AVG(Modelo_neg),2) as j,\nROUND(AVG(inversion),2) as k,\nROUND

In [ ]:
# Depronto con pivot

#pivot
#        (
#        MIN(a)
#        for Equipo_a IN ("Te lo cambio.", "Lo tengo y te lo presto.","Gestión de programas curriculares","Administración de Inmuebles","Servicios a domicilio","Gestión de Menús Ejecutivos","Gestión de programas curriculares","Deserción de estudiantes","Gestión de Residencias y Habitaciones Universitarias.")
#        ) as PV


select Equipo_evaluar as Equipo_a 
    from PROMEDIOS
    where (a = (
    select MIN(a) as min_a
    from PROMEDIOS
    ))
    order by a ASC

""").show()

In [ ]:
# 12. Consulte la lista de estudiantes con la correspondiente nota obtenida en la presentación 
# (nota promedio total de la evaluación obtenida por el equipo)
# Tu código a continuación...

In [39]:
ContextoSql.sql("""

WITH 

ASISTENCIAS AS (
select *,
substring(Equipo_perteneces,15,15) as llave_equipo_pertenece,
substring(Equipo_exponer,9,15) as llave_equipo_exponer
from VistaAsistencia
),

EVALUACION AS (
select *,
substring(Equipo_evaluar,1,15) as llave_Equipo_a_evaluar
from VistaEvaluacion
),

ASISTENTES_VERDADERAS AS (
select 
b1.Nombre as Nombre_a,
b1.Equipo_perteneces,
b2.*

from ASISTENCIAS b1 
inner join EVALUACION b2 on b1.Nombre = b2.Nombre
where b1.llave_equipo_pertenece != b2.llave_Equipo_a_evaluar 
)

    select Nombre_a,
    ROUND(AVG(Introduccion+Equipo+Problema+ventajas+Solucion+Producto+Traccion+Mercado+Competencia+Modelo_neg+inversion+contacto+Coordinacion_expo+Expo_claridad+Diapo_clara)/15,2) AS PROMEDIO_TOTAL
    from ASISTENTES_VERDADERAS
    group by Nombre_a
    order by Nombre_a

""").show()

+--------------------+--------------+
|            Nombre_a|PROMEDIO_TOTAL|
+--------------------+--------------+
| abanolc@unal.edu.co|          3.76|
|acastrillonv@unal...|          3.99|
| aflemag@unal.edu.co|          4.33|
|anapariciom@unal....|          4.12|
|angutierrezb@unal...|          4.22|
|anoriega@unal.edu.co|          4.62|
|auarbelaeza@unal....|          4.36|
| bocampo@unal.edu.co|          4.19|
|cgiraldo@unal.edu.co|          3.43|
|cjfunezg@unal.edu.co|          4.33|
|cquinchiar@unal.e...|           4.5|
| dadazam@unal.edu.co|          4.47|
|daestradam@unal.e...|          4.04|
|davgarciava@unal....|          4.22|
|dballesteroso@una...|          4.77|
|  dbrito@unal.edu.co|          4.66|
|dcadavid@unal.edu.co|          4.75|
|dcardonaal@unal.e...|          4.09|
|dchavarriar@unal....|          4.92|
|dgarciabl@unal.ed...|          4.31|
+--------------------+--------------+
only showing top 20 rows

